In [3]:
np.array([[0,1/6,5/6],[5/6,0,5/6],[1/6,1/6,0]])

array([[0.        , 0.16666667, 0.83333333],
       [0.83333333, 0.        , 0.83333333],
       [0.16666667, 0.16666667, 0.        ]])

In [1]:
import cvxpy as cp
import numpy as np
import mosek
from itertools import permutations
from math import sqrt,factorial,log
#from sympy import Matrix
import random 
import time
import pandas as pd
from scipy.linalg import svd,inv

In [2]:
def prefer(loci,locj):
    if loci==locj:
        val=1/2
    else:
        val=int(loci<locj)
        #val=2*int(loci<locj)-1
    return val

def prefermatrix(rank):
    loc=[rank.index(i) for i in range(n) ]
    a=np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            if i!=j:
                a[i,j]= prefer(loc[i],loc[j])
    return a

def FrobeniusNorm(a):
    norm=0
    (b,c)=a.shape
    for i in range(b):
        for j in range(c):
            norm+=(a[i,j])**2
    return sqrt(norm)

def matrix2vector(m):
    n=m.shape[0]
    Aj=np.zeros((int(n*(n-1)/2),1))
    index=0
    for i in range(1,n):
        for j in range(i):
            if i != j:
                Aj[index,0]=m[i,j]
                index+=1
    return Aj
def matrix2vector_long(m):
    #print(m.shape)
    n=m.shape[0]
    Aj=np.zeros((int(n*(n-1)),1))
    index=0
    for i in range(1,n):
        for j in range(n):
            if i != j:
                Aj[index,0]=m[i,j]
                index+=1
    return Aj

def entropy(xx):
    ent=0
    negative=0
    zero=0
    for i in range(T):
        if xx[i]<0:
            negative+=1
            continue
        if xx[i]==0:
            zero+=1
            continue
        ent-=xx[i]*log(xx[i])
        
    print('the number of negative values: ',negative,'; the number of 0: ',zero)
    #print('entropy is: ',ent[0])
    return ent #[0]

In [3]:
def mev(T,theoretical,empirical):
    x = cp.Variable(shape=(T,1))
    obj = cp.Maximize( cp.sum(cp.entr(x)) ) #cp.sum(cp.entr(x)) #cp.norm(sum(theoretical[i]*x[i]for i in range(2))-empirical, 'nuc')
    constraints = [x>=0,cp.sum(x)==1,cp.norm(sum(theoretical[i]*x[i]for i in range(T))-empirical,'fro')<=0.0001]
    prob = cp.Problem(obj, constraints)
    starttime = time.time()
    prob.solve(solver=cp.MOSEK, verbose=False)
    endtime = time.time()
    if prob.status not in ["infeasible", "unbounded"]:
        print('Row reduction method')
        return assess(prob.value,x.value,endtime-starttime,n,T)
    else:
        return [0]*4
    

In [4]:
def report(dist,theoretical,empirical,name):
    length=len(dist)
    result=sum(theoretical[i]*dist[i]for i in range(length))
    print('\nThe electoral system is: '+name)
    print('\nThe resulting pairwise preference is:')
    print(result)
    print('\nThe empirical pairwise preference is:')
    print(empirical)
    print('\nThe Frobenius norm between them is:')
    print(FrobeniusNorm(result-empirical))
    print('----------------------------')

In [5]:
def rowreduce(T,theoretical,empirical):
    b_tem=matrix2vector(empirical)
    A_tem=matrix2vector(theoretical[0])
    for k in range(1,T):
        A_tem=np.concatenate((A_tem,matrix2vector(theoretical[k])),axis=1)
    print(A_tem.shape)
    U, s, VT = svd(A_tem,full_matrices=True)
    Sigma = np.zeros((A_tem.shape[0], A_tem.shape[1]))
    # populate Sigma with n x n diagonal matrix
    Sigma[:A_tem.shape[0], :A_tem.shape[0]] = np.diag(s)
    A=Sigma.dot(VT)
    RA=np.linalg.matrix_rank(A)
    print(RA)
    b=inv(U).dot(b_tem)
    B=U.dot(A)
    print(np.allclose(A_tem,B))
    A=A[0:RA,:]
    b=b[0:RA]
    
    return A,b

In [6]:
def rowreduce_long(T,theoretical,empirical):
    b_tem=matrix2vector_long(empirical)
    A_tem=matrix2vector_long(theoretical[0])
    for k in range(1,T):
        A_tem=np.concatenate((A_tem,matrix2vector_long(theoretical[k])),axis=1)
    print(A_tem.shape)
    U, s, VT = svd(A_tem,full_matrices=True)
    Sigma = np.zeros((A_tem.shape[0], A_tem.shape[1]))
    # populate Sigma with n x n diagonal matrix
    Sigma[:A_tem.shape[0], :A_tem.shape[0]] = np.diag(s)
    A=Sigma.dot(VT)
    RA=np.linalg.matrix_rank(A)
    print(RA)
    b=inv(U).dot(b_tem)
    B=U.dot(A)
    print(np.allclose(A_tem,B))
    A=A[0:RA,:]
    b=b[0:RA]
    
    return A,b

In [7]:
def mev_rowcolreduce_long(T,theoretical,empirical):
    b_tem=matrix2vector_long(empirical)
    A_tem=matrix2vector_long(theoretical[0])
    for k in range(1,T):
        A_tem=np.concatenate((A_tem,matrix2vector_long(theoretical[k])),axis=1)
    print(A_tem.shape)
    U, s, VT = svd(A_tem,full_matrices=True)
    Sigma = np.zeros((A_tem.shape[0], A_tem.shape[1]))
    # populate Sigma with n x n diagonal matrix
    Sigma[:A_tem.shape[0], :A_tem.shape[0]] = np.diag(s)
    #A=Sigma.dot(VT)
    RA=np.linalg.matrix_rank(A_tem) #RA==n(n-1)/2
    b=inv(U).dot(b_tem)

    h=np.linalg.pinv(Sigma).dot(b)

    x = cp.Variable(shape=(T,1))
    y = cp.Variable(shape=(T,1))
    q = cp.Variable(shape=(T-RA,1))
    obj = cp.Maximize( cp.sum(cp.entr(x))-5*cp.sum_squares(q) )#cp.Maximize( cp.sum(cp.entr(h+A_new@y)) ) #cp.sum(cp.entr(x)) #cp.norm(sum(theoretical[i]*x[i]for i in range(2))-empirical, 'nuc')
    constraints = [VT[RA:,:]@x+q==y[RA:],sum(x)==1,x>=0,y[0:RA]==h[0:RA]]#+[np.zeros((1,T))@(A_new@y)>=1-sum(h),np.zeros((1,T))@(A_new@y)<=1-sum(h)]
    prob = cp.Problem(obj, constraints)
    starttime = time.time()
    prob.solve(solver=cp.MOSEK, verbose=False)
    endtime = time.time()

    # Print result.
    if prob.status not in ["infeasible", "unbounded"]:
        print('Row reduction method')
        return assess(prob.value,x.value,endtime-starttime,n,T)
    else:
        return [0]*4

In [8]:
for n in range(3,7):
    T=factorial(n)
    # Get all permutations
    perm=permutations(range(n))
    # distRD[t] Gives the proportion of people who vote for ordering t.
    start=1
    dist=[0]*T
    for i in range(T-1):
        dist[i]=random.uniform(0,start)
        start=1-sum(dist)
        if start<10**(-8):
            dist[i+1]=start
            break    
    empirical=np.zeros((n,n))
    theoretical={}
    k=0
    for i in list(perm):
        empirical+=prefermatrix(i)*dist[k]
        theoretical[k]=prefermatrix(i)
        k+=1

    b_tem=matrix2vector_long(empirical)
    A_tem=matrix2vector_long(theoretical[0])
    for k in range(1,T):
        A_tem=np.concatenate((A_tem,matrix2vector_long(theoretical[k])),axis=1)
    print(A_tem.shape)
    print(np.linalg.matrix_rank(A_tem))

(6, 6)
4
(12, 24)
7
(20, 120)
11
(30, 720)
16


In [19]:
n=4
T=factorial(n)
# Get all permutations
perm=permutations(range(n))
# distRD[t] Gives the proportion of people who vote for ordering t.
start=1
dist=[0]*T
for i in range(T-1):
    dist[i]=round(random.uniform(0,start),2)
    start=1-sum(dist)
    if start<10**(-3):
        dist[i+1]=start
        break   
random.shuffle(dist)
empirical=np.zeros((n,n))
theoretical={}
k=0
for i in list(perm):
    empirical+=prefermatrix(i)*dist[k]
    theoretical[k]=prefermatrix(i)
    k+=1
    #print(i)
product=[1]*n
for i in range(n):
    for j in range(n):
        if i != j:
            product[i]=product[i]*empirical[i,j]
print(empirical)
print(sum(product))

X=[]
T_tem=int(T/n)
runtime=0
for c in range(n):
    if product[c]==0:
        candidates=[*range(n)]
        candidates.remove(c)
        emp_tem=np.delete(empirical, c, 0)
        emp_tem=np.delete(emp_tem, c, 1)
        the_tem={}
        for k in range(T_tem):
            the_tem[k]=np.delete(theoretical[c*T_tem+k], c, 0)
            the_tem[k]=np.delete(the_tem[k], c, 1)
        #T=int(T/n)
        #n=n-1

        b=matrix2vector_long(emp_tem)
        A=matrix2vector_long(the_tem[0])
        for k in range(1,T_tem):
            A=np.concatenate((A,matrix2vector_long(the_tem[k])),axis=1)
        x = cp.Variable(shape=(T_tem,1))
        obj = cp.Maximize( cp.sum(cp.entr(x)) ) #cp.sum(cp.entr(x)) #cp.norm(sum(theoretical[i]*x[i]for i in range(2))-empirical, 'nuc')
        constraints = [x>=0,cp.sum(x)==1,A@x==b]
        prob = cp.Problem(obj, constraints)

        starttime = time.time()
        prob.solve(solver=cp.MOSEK, verbose=False)
        endtime = time.time()
        runtime=endtime-starttime
        # Print result.
        if prob.status not in ["infeasible", "unbounded"]:
            xx=x.value.T*product[c]
            xx=xx.tolist()[0]
            X+=xx
            #print('Row reduction method')
            #return #assess(prob.value,x.value,endtime-starttime,n,T)
    else:
        X+=[0]*T_tem

assess(0,np.array(X),runtime,n,T)

[[0.   0.46 0.53 0.6 ]
 [0.54 0.   0.51 0.56]
 [0.47 0.49 0.   0.62]
 [0.4  0.44 0.38 0.  ]]
0.5101700000000001
The optimal value is: 0
Run time is: 0
the number of negative values:  0 ; the number of 0:  24
The Frobenius Norm of the difference is: 1.1183022847155417
entropy is:  0


[0, 0, 1.1183022847155417, 0]

In [10]:
a=[*range(4)]
a.remove(3)
a

[0, 1, 2]

In [11]:
empirical

array([[0.  , 0.23, 0.75, 0.21],
       [0.77, 0.  , 0.69, 0.09],
       [0.25, 0.31, 0.  , 0.32],
       [0.79, 0.91, 0.68, 0.  ]])

In [10]:
def rowreduce_non(T,theoretical,empirical):
    b_tem=matrix2vector(empirical)
    A_tem=matrix2vector(theoretical[0])
    for k in range(1,T):
        A_tem=np.concatenate((A_tem,matrix2vector(theoretical[k])),axis=1)

    B_tem=Matrix(np.concatenate((A_tem,b_tem),axis=1))
    B=B_tem.echelon_form()
    B=np.array(B).astype(np.float64)
    print(B)
    RB=np.linalg.matrix_rank(B)
    print(RB)
    A=B[0:RB,0:-1]
    b=B[0:RB,T:T+1]
    return A,b

In [11]:
def mev_rowreduce_non(T,theoretical,empirical):
    b=matrix2vector(empirical)
    A=matrix2vector(theoretical[0])
    for k in range(1,T):
        A=np.concatenate((A,matrix2vector(theoretical[k])),axis=1)
    x = cp.Variable(shape=(T,1))
    obj = cp.Maximize( cp.sum(cp.entr(x)) ) #cp.sum(cp.entr(x)) #cp.norm(sum(theoretical[i]*x[i]for i in range(2))-empirical, 'nuc')
    constraints = [x>=0,cp.sum(x)==1,A@x==b]
    prob = cp.Problem(obj, constraints)

    starttime = time.time()
    prob.solve(solver=cp.MOSEK, verbose=False)
    endtime = time.time()
    # Print result.
    if prob.status not in ["infeasible", "unbounded"]:
        print('Row reduction method')
        return assess(prob.value,x.value,endtime-starttime,n,T)
    else:
        return [0]*4

In [20]:
def mev_rowreduce_JL(T,theoretical,empirical):
    b=matrix2vector(empirical)
    A=matrix2vector(theoretical[0])
    for k in range(1,T):
        A=np.concatenate((A,matrix2vector(theoretical[k])),axis=1)
    x = cp.Variable(shape=(T,1))
    obj = cp.Maximize( cp.sum(cp.entr(x)) ) #cp.sum(cp.entr(x)) #cp.norm(sum(theoretical[i]*x[i]for i in range(2))-empirical, 'nuc')
    constraints = [x>=0,cp.sum(x)==1,A@x==b]
    prob = cp.Problem(obj, constraints)

    starttime = time.time()
    prob.solve(solver=cp.MOSEK, verbose=False)
    endtime = time.time()
    # Print result.
    if prob.status not in ["infeasible", "unbounded"]:
        print('Row reduction method')
        return assess(prob.value,x.value,endtime-starttime,n,T)
    else:
        return [0]*4

In [12]:
def mev_rowreduce_non_long(T,theoretical,empirical):
    b=matrix2vector_long(empirical)
    A=matrix2vector_long(theoretical[0])
    for k in range(1,T):
        A=np.concatenate((A,matrix2vector_long(theoretical[k])),axis=1)
    x = cp.Variable(shape=(T,1))
    obj = cp.Maximize( cp.sum(cp.entr(x)) ) #cp.sum(cp.entr(x)) #cp.norm(sum(theoretical[i]*x[i]for i in range(2))-empirical, 'nuc')
    constraints = [x>=0,cp.sum(x)==1,A@x==b]
    prob = cp.Problem(obj, constraints)

    starttime = time.time()
    prob.solve(solver=cp.MOSEK, verbose=False)
    endtime = time.time()
    # Print result.
    if prob.status not in ["infeasible", "unbounded"]:
        print('Row reduction method')
        return assess(prob.value,x.value,endtime-starttime,n,T)
    else:
        return [0]*4

In [13]:
def mev_rowreduce(T,theoretical,empirical):
    A,b=rowreduce(T,theoretical,empirical)
    x = cp.Variable(shape=(T,1))
    obj = cp.Maximize( cp.sum(cp.entr(x)) ) #cp.sum(cp.entr(x)) #cp.norm(sum(theoretical[i]*x[i]for i in range(2))-empirical, 'nuc')
    constraints = [x>=0,cp.sum(x)==1,A@x==b]
    prob = cp.Problem(obj, constraints)

    starttime = time.time()
    prob.solve(solver=cp.MOSEK, verbose=False)
    endtime = time.time()
    # Print result.
    if prob.status not in ["infeasible", "unbounded"]:
        print('Row reduction method')
        return assess(prob.value,x.value,endtime-starttime,n,T)
    else:
        return [0]*4

In [14]:
def mev_rowreduce_long(T,theoretical,empirical):
    A,b=rowreduce_long(T,theoretical,empirical)
    x = cp.Variable(shape=(T,1))
    obj = cp.Maximize( cp.sum(cp.entr(x)) ) #cp.sum(cp.entr(x)) #cp.norm(sum(theoretical[i]*x[i]for i in range(2))-empirical, 'nuc')
    constraints = [x>=0,cp.sum(x)==1,A@x==b]
    prob = cp.Problem(obj, constraints)

    starttime = time.time()
    prob.solve(solver=cp.MOSEK, verbose=False)
    endtime = time.time()
    # Print result.
    if prob.status not in ["infeasible", "unbounded"]:
        print('Row reduction method')
        return assess(prob.value,x.value,endtime-starttime,n,T)
    else:
        return [0]*4

In [15]:
def assess(probvalue,xvalue,runtime,n,T):
    print("The optimal value is:", probvalue)
    print('Run time is:',runtime)
    #print('The optimal solution is:',xvalue)
    diff=np.zeros((n,n))
    for i in range(1,n):
        for j in range(i):
            diff[i,j]=sum(theoretical[t][i,j]*xvalue[t] for t in range(T))-empirical[i,j]
    norm=FrobeniusNorm(diff)
    ent=entropy(xvalue)
    print('The Frobenius Norm of the difference is:',norm)
    print('entropy is: ',ent)
    return [probvalue,runtime,norm,ent]

In [16]:
n=4
T=factorial(n)
# Get all permutations
perm=permutations(range(n))
# distRD[t] Gives the proportion of people who vote for ordering t.
start=1
dist=[0]*T
for i in range(T-1):
    dist[i]=round(random.uniform(0,start),2)
    start=1-sum(dist)
    if start<10**(-3):
        dist[i+1]=start
        break    
empirical=np.zeros((n,n))
theoretical={}
k=0
for i in list(perm):
    empirical+=prefermatrix(i)*dist[k]
    theoretical[k]=prefermatrix(i)
    k+=1
    #print(i)
    
def mev_removeone(T,theoretical,empirical):
    product=[1]*n
    for i in range(n):
        for j in range(n):
            if i != j:
                product[i]=product[i]*empirical[i,j]
    #print(empirical)
    print(sum(product))

    X=[]
    runtime=0
    T_tem=int(T/n)
    for c in range(n):
        if product[c]<=10**(-8):
            candidates=[*range(n)]
            candidates.remove(c)
            emp_tem=np.delete(empirical, c, 0)
            emp_tem=np.delete(emp_tem, c, 1)
            the_tem={}
            for k in range(T_tem):
                the_tem[k]=np.delete(theoretical[c*T_tem+k], c, 0)
                the_tem[k]=np.delete(the_tem[k], c, 1)
            #T=int(T/n)
            #n=n-1

            b=matrix2vector_long(emp_tem)
            A=matrix2vector_long(the_tem[0])
            for k in range(1,T_tem):
                A=np.concatenate((A,matrix2vector_long(the_tem[k])),axis=1)
            x = cp.Variable(shape=(T_tem,1))
            obj = cp.Maximize( cp.sum(cp.entr(x)) ) #cp.sum(cp.entr(x)) #cp.norm(sum(theoretical[i]*x[i]for i in range(2))-empirical, 'nuc')
            constraints = [x>=0,cp.sum(x)==1,A@x==b]
            prob = cp.Problem(obj, constraints)

            starttime = time.time()
            prob.solve(solver=cp.MOSEK, verbose=False)
            endtime = time.time()
            # Print result.
            if prob.status not in ["infeasible", "unbounded"]:
                xx=x.value.T*product[c]
                xx=xx.tolist()[0]
                X+=xx
                runtime+=endtime-starttime
                #print('Row reduction method')
                #return #assess(prob.value,x.value,endtime-starttime,n,T)
        else:
            X+=[0]*T_tem
    #print('product: ',product)
    #print(X)
    return assess(0,np.array(X).T,runtime,n,T)

In [17]:
df=pd.DataFrame(columns=['probvalue','runtime','norm','entropy','met'],index=[*range(5*3)])
row=0
for r in range(3):
    for n in range(4,7):
        T=factorial(n)
        # Get all permutations
        perm=permutations(range(n))
        # distRD[t] Gives the proportion of people who vote for ordering t.
        start=1
        dist=[0]*T
        for i in range(T-1):
            dist[i]=round(random.uniform(0,start),4)
            start=1-sum(dist)
            if start<10**(-8):
                dist[i+1]=start
                break    
        random.shuffle(dist)
        print(dist)
        empirical=np.zeros((n,n))
        theoretical={}
        k=0
        for i in list(perm):
            empirical+=prefermatrix(i)*dist[k]
            theoretical[k]=prefermatrix(i)
            k+=1
        #df.loc[row,:]=mev_rowcolreduce(T,theoretical,empirical)+['reducerowcol']
        #df.loc[row,:]=mev_rowreduce_long(T,theoretical,empirical)+['reducerow_long']
        df.loc[row,:]=mev_rowreduce_non_long(T,theoretical,empirical)+['nonreducerow_long']
        df.loc[row+1,:]=mev_rowreduce_non(T,theoretical,empirical)+['nonreducerow_short']
        df.loc[row+2,:]=mev(T,theoretical,empirical)+['origin']
        row+=3
        
df=df.drop(['probvalue'], axis=1)

[0, 0, 0.002, 0, 0, 0.0051, 0.0001, 0, 0.2694, 0.0264, 0.2153, 0.0075, 0, 0.0023, 0.0792, 0.0003, 0.0001, 0.0079, 0.0009, 0, 0, 0, 0.3835, 1.1102230246251565e-16]
Row reduction method
The optimal value is: 2.831179171518885
Run time is: 0.12729215621948242
the number of negative values:  0 ; the number of 0:  0
The Frobenius Norm of the difference is: 9.200032972234141e-09
entropy is:  [2.83117917]
Row reduction method
The optimal value is: 2.8311791725081257
Run time is: 0.017586708068847656
the number of negative values:  0 ; the number of 0:  0
The Frobenius Norm of the difference is: 8.510224309058558e-09
entropy is:  [2.83117917]
Row reduction method
The optimal value is: 2.831360060070306
Run time is: 0.028450727462768555
the number of negative values:  0 ; the number of 0:  0
The Frobenius Norm of the difference is: 7.070960733352016e-05
entropy is:  [2.83136006]
[0, 0, 0, 0.6327, 0, 0, 0, 0, 0, 0.0856, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

C:\Users\zhouq\anaconda3\lib\site-packages\pandas\core\internals\blocks.py:849: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arr_value = np.array(value)


Row reduction method
The optimal value is: 2.4589197262067866
Run time is: 0.05547928810119629
the number of negative values:  0 ; the number of 0:  60
The Frobenius Norm of the difference is: 3.491287275007844e-10
entropy is:  [2.45891973]
Row reduction method
The optimal value is: 2.4589197269580527
Run time is: 0.02744770050048828
the number of negative values:  0 ; the number of 0:  60
The Frobenius Norm of the difference is: 3.1480786719649e-10
entropy is:  [2.45891973]
Row reduction method
The optimal value is: 2.459961895038311
Run time is: 0.06485772132873535
the number of negative values:  0 ; the number of 0:  0
The Frobenius Norm of the difference is: 7.071086503520818e-05
entropy is:  [2.4599619]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.1102230246251565e-16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Row reduction method
The optimal value is: -inf
Run time is: 0.4296398162841797
the number of negative values:  1 ; the number of 0:  0
The Frobenius Norm of the difference is: 7.071109741177459e-05
entropy is:  [0.68617018]
[1.1102230246251565e-16, 0.0, 0.0024, 0, 0.2811, 0.0, 0.5663, 0.0624, 0.0001, 0.0001, 0.006, 0.0666, 0.0003, 0.0007, 0.0011, 0.0004, 0.0002, 0.0123, 0, 0.0, 0, 0, 0, 0]
Row reduction method
The optimal value is: 1.6983138993519913
Run time is: 0.019939899444580078
the number of negative values:  0 ; the number of 0:  0
The Frobenius Norm of the difference is: 7.0535230413818575e-09
entropy is:  [1.6983139]
Row reduction method
The optimal value is: 1.6983138996640987
Run time is: 0.019946813583374023
the number of negative values:  0 ; the number of 0:  0
The Frobenius Norm of the difference is: 3.0331034074846772e-09
entropy is:  [1.6983139]
Row reduction method
The optimal value is: 1.6988035047204022
Run time is: 0.0585627555847168
the number of negative values:

In [18]:
df

,runtime,norm,entropy,met
0,0.127292,9.20003e-09,[2.831179171518885],nonreducerow_long
1,0.0175867,8.51022e-09,[2.8311791725081266],nonreducerow_short
2,0.0284507,7.07096e-05,[2.831360060070306],origin
3,0.0554793,3.49129e-10,[2.458919726206786],nonreducerow_long
4,0.0274477,3.14808e-10,[2.4589197269580514],nonreducerow_short
5,0.0648577,7.07109e-05,[2.4599618950383104],origin
6,0.0963535,3.99577e-11,[1.9194112228984612],nonreducerow_long
7,0.0857458,4.62107e-11,[1.9194112228363922],nonreducerow_short
8,0.434498,7.07107e-05,[1.920031837378617],origin
9,0.0170898,2.63511e-09,[1.8175409198069663],nonreducerow_long


In [8]:
candidate=['A','B','C']
n=len(candidate)

# Get all permutations
perm=permutations(candidate)
distRD=[0.99,0,0,0,0,0.01]
#distRD=[0.5,0,0,0,0,0.5]
#distRD=[0.5,0,0,0,0,0.5]
distSRD=[0.25,0.25,0,0,0.25,0.25]
#[1/n]*len(list(perm))

empirical=np.zeros((n,n))
theoretical={}
index=0
for i in list(perm):
    #print(i)
    empirical+=prefermatrix(i)*distRD[index]
    theoretical[index]=prefermatrix(i)
    index+=1

#x=mev(index,theoretical,empirical)



The optimal value is: 0.12631801147802643

The optimal solution is:
[9.79925444e-01 5.03724098e-03 5.03724098e-03 4.98731526e-03
 4.98731526e-03 2.54438587e-05]


In [9]:
def assess(xx,n):
    outcome=np.zeros((n,n))
    for i in range(1,n):
        for j in range(i):
            outcome[i,j]=sum(theoretical[t][i,j]*xx[t] for t in range(T))
            outcome[j,i]=1-outcome[i,j]
    print('The output matrix is:\n',outcome)

In [10]:
empirical1=np.array([[0,1/6,5/6],[5/6,0,5/6],[1/6,1/6,0]])
x1=mev(index,theoretical,empirical1)
sum(theoretical[i]*x1[i]for i in range(6))


The optimal value is: 1.1085892965204331

The optimal solution is:
[0.0787927  0.07874272 0.67574792 0.07879271 0.00918125 0.0787427 ]


array([[0.        , 0.16671667, 0.83328333],
       [0.83328333, 0.        , 0.83333333],
       [0.16671667, 0.16666667, 0.        ]])

In [12]:
ab=0.4
bc=0.2
ac=0.8
empirical2=np.array([[0,ab,ac],[1-ab,0,bc],[1-ac,1-bc,0]])
x2=mev(index,theoretical,empirical2)
sum(theoretical[i]*x2[i]for i in range(6))


The optimal value is: -inf

The optimal solution is:
None


TypeError: 'NoneType' object is not subscriptable

In [46]:
empirical3=np.array([[0,0,1],[1,0,1],[0,0,0]])
x3=mev(index,theoretical,empirical3)
sum(theoretical[i]*x3[i]for i in range(6))


The optimal value is: 0.0011128352587590298

The optimal solution is:
[4.41957634e-05 5.24881040e-06 9.99901109e-01 4.41952675e-05
 1.88125592e-09 5.24881031e-06]


array([[0.00000000e+00, 4.94464551e-05, 9.99950554e-01],
       [9.99950554e-01, 0.00000000e+00, 9.99989500e-01],
       [4.94459590e-05, 1.04995020e-05, 0.00000000e+00]])

In [8]:
report(x,theoretical,empirical,'MEV0')
report(distSRD,theoretical,empirical,'SRD')


The electoral system is: MEV0

The resulting pairwise preference is:
[[0.         0.98995    0.98999993]
 [0.01005    0.         0.98995   ]
 [0.01000007 0.01005    0.        ]]

The empirical pairwise preference is:
[[0.   0.99 0.99]
 [0.01 0.   0.99]
 [0.01 0.01 0.  ]]

The Frobenius norm between them is:
0.00010000026908275112
----------------------------

The electoral system is: SRD

The resulting pairwise preference is:
[[0.   0.75 0.5 ]
 [0.25 0.   0.25]
 [0.5  0.75 0.  ]]

The empirical pairwise preference is:
[[0.   0.99 0.99]
 [0.01 0.   0.99]
 [0.01 0.01 0.  ]]

The Frobenius norm between them is:
1.3002307487519282
----------------------------


In [12]:
theoretical

{0: array([[0., 1., 1.],
        [0., 0., 1.],
        [0., 0., 0.]]),
 1: array([[0., 1., 1.],
        [0., 0., 0.],
        [0., 1., 0.]]),
 2: array([[0., 0., 1.],
        [1., 0., 1.],
        [0., 0., 0.]]),
 3: array([[0., 0., 0.],
        [1., 0., 1.],
        [1., 0., 0.]]),
 4: array([[0., 1., 0.],
        [0., 0., 0.],
        [1., 1., 0.]]),
 5: array([[0., 0., 0.],
        [1., 0., 0.],
        [1., 1., 0.]])}

# two candidates

In [8]:
candidate=['c1','c2']
n=len(candidate)

# Get all permutations
perm=permutations(candidate)
distRD=[0.6,0.4]
#[1/n]*len(list(perm))

empirical=np.zeros((n,n))
theoretical={}
index=0
for i in list(perm):
    #print(i)
    empirical+=prefermatrix(i)*distRD[index]
    theoretical[index]=prefermatrix(i)
    index+=1
    
x=mev(index,theoretical,empirical)


The optimal value is: 0.6730403239642286

The optimal solution is:
[0.5999293 0.4000707]


In [14]:
report(x,theoretical,empirical,'MEV0')
report(distSRD,theoretical,empirical,'SRD')


The electoral system is: MEV0

The resulting pairwise preference is:
[[0.         0.98995    0.98999993]
 [0.01005    0.         0.98995   ]
 [0.01000007 0.01005    0.        ]]

The empirical pairwise preference is:
[[0.   0.99 0.99]
 [0.01 0.   0.99]
 [0.01 0.01 0.  ]]

The Frobenius norm between them is:
0.00010000026908275112
----------------------------

The electoral system is: SRD

The resulting pairwise preference is:
[[0.   0.75 0.5 ]
 [0.25 0.   0.25]
 [0.5  0.75 0.  ]]

The empirical pairwise preference is:
[[0.   0.99 0.99]
 [0.01 0.   0.99]
 [0.01 0.01 0.  ]]

The Frobenius norm between them is:
1.3002307487519282
----------------------------


In [15]:
maxval=0
minval=0.5
for i in range(100):
    r=0.01*i+0.09
    distRD=[r,0,0.99-r,0,0,0.01]
    
    candidate=['c1','c2','c3']
    n=len(candidate)

    # Get all permutations
    perm=permutations(candidate)
    empirical=np.zeros((n,n))
    theoretical={}
    index=0
    for i in list(perm):
        empirical+=prefermatrix(i)*distRD[index]
        theoretical[index]=prefermatrix(i)
        index+=1
    
    x=mev(index,theoretical,empirical)
    if maxval < x[4]+x[5]:
        maxval=x[4]+x[5]
    if minval > x[4]+x[5]:
        minval=x[4]+x[5]


The optimal value is: 0.390588304690433

The optimal solution is:
[8.09940173e-02 8.94757237e-03 9.00000699e-01 8.97766378e-03
 8.85221305e-05 9.91525500e-04]

The optimal value is: 0.41402488044481206

The optimal solution is:
[9.09005561e-02 9.03670074e-03 8.90005014e-01 9.06534610e-03
 9.14067690e-05 9.00976526e-04]

The optimal value is: 0.43625631014493893

The optimal solution is:
[1.00822410e-01 9.11082864e-03 8.80009058e-01 9.13812995e-03
 9.40787459e-05 8.25494434e-04]

The optimal value is: 0.45738786881749255

The optimal solution is:
[1.10756155e-01 9.17346505e-03 8.70012738e-01 9.19951189e-03
 9.64427665e-05 7.61687554e-04]

The optimal value is: 0.4775059989774309

The optimal solution is:
[1.20699271e-01 9.22705897e-03 8.60016117e-01 9.25193618e-03
 9.85721444e-05 7.07044500e-04]

The optimal value is: 0.4966828627963986

The optimal solution is:
[1.30649779e-01 9.27312360e-03 8.50019651e-01 9.29688964e-03
 1.00884036e-04 6.59671965e-04]

The optimal value is: 0.5149795


The optimal value is: 0.7570933149055215

The optimal solution is:
[6.20089489e-01 9.64559350e-03 3.60218734e-01 9.63868074e-03
 2.57998500e-04 1.49504350e-04]

The optimal value is: 0.7514416153123359

The optimal solution is:
[6.30086170e-01 9.64124822e-03 3.50226741e-01 9.63377388e-03
 2.65100806e-04 1.46966934e-04]

The optimal value is: 0.7453459194929788

The optimal solution is:
[6.40082898e-01 9.63645015e-03 3.40235161e-01 9.62840705e-03
 2.72600871e-04 1.44482443e-04]

The optimal value is: 0.7388003355406929

The optimal solution is:
[6.50079685e-01 9.63116832e-03 3.30244018e-01 9.62254858e-03
 2.80518809e-04 1.42061740e-04]

The optimal value is: 0.731798390586217

The optimal solution is:
[6.60076511e-01 9.62538492e-03 3.20253346e-01 9.61618047e-03
 2.88890328e-04 1.39686831e-04]

The optimal value is: 0.7243329864374887

The optimal solution is:
[6.70073392e-01 9.61903105e-03 3.10263202e-01 9.60923228e-03
 2.97768528e-04 1.37373726e-04]

The optimal value is: 0.7163963494

TypeError: 'NoneType' object is not subscriptable

In [16]:
maxval

0.00992490907677908

In [17]:
minval

0.00037970086568300614